In [1]:
import os
import datetime
import shutil
from pathlib import Path
from collections import Counter
from string import Template

import yaml
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

from ultralytics import YOLO
import torch
from roboflow import Roboflow

import warnings
warnings.filterwarnings('ignore')

In [2]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=M4ZXOEAPZ93B0QTCMCDR80GRKF8GGG
%env CLEARML_API_SECRET_KEY=7bKMaA2rcQeFy7v_776se4TEdL65HWYgpK0NAAj2CF3sYNBEZO4OCTu_pkehS9xRLQo

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=M4ZXOEAPZ93B0QTCMCDR80GRKF8GGG
env: CLEARML_API_SECRET_KEY=7bKMaA2rcQeFy7v_776se4TEdL65HWYgpK0NAAj2CF3sYNBEZO4OCTu_pkehS9xRLQo


In [3]:
API_KEY = "UDTU0z0V2s0YrtieNgro"
WORKSPACE = "brad-dwyer"
PROJECT = "wildfire-smoke"
VERSION = 1

rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT)
version = project.version(VERSION)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


In [4]:
import clearml
from clearml import Task

clearml.browser_login()

In [5]:
def train_model(batch, epochs, img_size=640):
    """
    Args:
        hyperparams to tune
        batch = [16, 32, 64]
        img_size = [256, 512, 640]
    """
    
    # Define parameters
    dataset_yaml = f'Wildfire-Smoke-1/data.yaml'
    project = f'wildfire_detect'
    name = f"wildfire_detect"
    lr = 1e-3
    optimizer = 'AdamW'
    
    # create ClearML Task
    hyp_task = Task.init(
        project_name=project, 
        task_name=name
    )
    
    model_variant = 'yolo11n.pt'
    hyp_task.set_parameter("model_variant", model_variant)
    model = YOLO(model_variant, task='detect')

    # put all argument in the dict and pass it to ClearML
    args = dict(
        data=dataset_yaml,
        epochs=epochs, 
        batch=batch, 
        project=project,
        name=name,
        lr0=lr,
        imgsz=img_size,
        optimizer=optimizer,
        cache=True
    )
    hyp_task.connect(args)

    model.train(**args)
#         results[k] = model.metrics
    
    hyp_task.close()

In [6]:
train_model(batch=16, epochs=20)

ClearML Task: overwriting (reusing) task id=16818def0b99409f9e5de51fef703c77
2025-01-24 17:01:07,611 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2025-01-24 17:01:07,612 - clearml.Repository Detection - WARNING - Please install nbconvert using "pip install nbconvert"
2025-01-24 17:01:07,676 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/41569bcfc9b94171a716988405e5c998/experiments/16818def0b99409f9e5de51fef703c77/output/log
2025-01-24 17:01:21,065 - clearml.model - INFO - Selected model id: 40ea04288b84450d90999c739ed0f337
Ultralytics 8.3.66 🚀 Python-3.11.10 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=Wildfire-Smoke-1/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=wildfire_detec

train: Scanning /mnt/c/Users/hi/work/kecilin_test/notebook/Wildfire-Smoke-1/train/labels.cache... 516 images, 0 backgrounds, 0 corrupt: 100%|██████████| 516/516 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.4GB RAM): 100%|██████████| 516/516 [00:00<00:00, 809.27it/s]
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.1 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all        147        147      0.846       0.85      0.889       0.48



20 epochs completed in 0.044 hours.
Optimizer stripped from wildfire_detect/wildfire_detect10/weights/last.pt, 5.5MB
Optimizer stripped from wildfire_detect/wildfire_detect10/weights/best.pt, 5.5MB

Validating wildfire_detect/wildfire_detect10/weights/best.pt...
Ultralytics 8.3.66 🚀 Python-3.11.10 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.15it/s]


                   all        147        147      0.835      0.878      0.909      0.494
Speed: 0.3ms preprocess, 2.8ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to wildfire_detect/wildfire_detect10
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'


█████████████████████████████████▉ 100% | 5.20/5.2 MB [00:01<00:00,  4.57MB/s]: 
